# Random Numbers

Generating random numbers is a very old computer science problem with
many useful applications. Here we will study how to use the rng (random
number generators) in c++, and then use to sample regions, compute
integral, and perform simulations.

Go to <https://tinyurl.com/ProgCPP-presentations> for a presentation regarding the topic.

## Linear congruential Random Number Generator

Represented by 
\begin{equation}
x' = (ax + c) \mod m,
\end{equation}

where $a, c, m$ are integers constants. Using $x = 1$, generate 100
numbers after iterating the same equation. Use
$a = 1664525, c = 1013904223, m=4294967296$. There are several proposal for the constants. For the
whole history, see
<https://en.wikipedia.org/wiki/Linear_congruential_generator> .

<div style="background-color: #f0f7fb; color: #055781; padding: 10px; border-left: 5px solid #055781; font-weight: bold; font-size: 20px; margin: 10px 0;">EXERCISE</div>

Let's implement the LCRNG and check, graphically, if there
are correlations, by plotting triplets of succesive random points. You
should:

1.  Generate N = 10000 points of the form $(x_i, x_{i+1}, x_{i+2})$,
    where $x_{i+1} = (a x_i + c) \mod m$. In toal you will need to call
    the LCRNG $3N$ times.

2.  Normalize by dividing by $m-1$ so the points are normalized to
    $[0, 1)$.

3.  Plot it. For gnuplot you can use the command:

    ``` gnuplot
    splot 'data.txt' w points ps 0.1 pt 2
    ```

4.  Now to the same but with constants $m=128, a=37, c=0$.

## Randon number generators from C++

```c++
#include <iostream>
#include <fstream>
#include <random>
#include <cstdlib>
#include <vector>

int main(int argc, char **argv)
{
  if (5 != argc) {
    std::cerr << "Error. Usage: \n" << argv[0] << " SEED SAMPLES A B\n";
    return 1;
  }
  const int SEED = std::atoi(argv[1]);;
  const int SAMPLES = std::atoi(argv[2]);
  const int A = std::atof(argv[3]);;
  const int B = std::atof(argv[4]);

  std::mt19937 gen(SEED);
  std::uniform_real_distribution<double> dist(A, B);
  std::ofstream fout("data.txt");
  for (int ii = 0; ii < SAMPLES; ++ii) {
    double r = dist(gen);
    fout << r << "\n";
  }
  fout.close();

  return 0;
}

```

### Computing the histogram

```c++
#include <iostream>
#include <fstream>
#include <random>
#include <cstdlib>
#include <vector>

int main(int argc, char **argv)
{
  if (8 != argc) {
    std::cerr << "Error. Usage:\n" << argv[0] << " SEED SAMPLES A B XMIN XMAX NBINS\n";
    return 1;
  }
  const int SEED = std::atoi(argv[1]);
  const int SAMPLES = std::atoi(argv[2]);
  const int A = std::atof(argv[3]);
  const int B = std::atof(argv[4]);
  const double XMIN = std::atof(argv[5]);
  const double XMAX = std::atof(argv[6]);
  const int NBINS = std::atoi(argv[7]);
  const double DX = (XMAX-XMIN)/NBINS;

  std::vector<double> histo(NBINS, 0.0);
  std::mt19937 gen(SEED);
  //std::uniform_real_distribution<double> dist(A, B);
  std::normal_distribution<double> dist(A, B);
  std::ofstream fout("data.txt");
  for (int ii = 0; ii < SAMPLES; ++ii) {
    double r = dist(gen);
    fout << r << "\n";
    int bin = int((r - XMIN)/DX); // compute the bin where the sample lies
    if (0 <= bin && bin < NBINS) { // check if the bin is included
      histo[bin]++; // increase the counter in that bin
    }
  }
  fout.close();

  fout.open("histo.txt");
  for (int ii = 0; ii < NBINS; ii++) {
    fout << XMIN + ii*DX << "\t" << histo[ii]/(DX*SAMPLES) << "\n";
  }
  fout.close();

  return 0;
}

```

## Generating numbers with a given distribution

```c++
#include <iostream>
#include <fstream>
#include <random>
//#include <cstdlib>
#include <vector>

double f(double x) {
  return 3.0*(1-x*x)/4.0;
}

int main(int argc, char **argv)
{
  const int SEED = std::atoi(argv[1]);;
  const int SAMPLES = std::atoi(argv[2]);
  const double XMIN = -2.0;
  const double XMAX = 2.0;
  const int NBINS = 50;
  const double DX = (XMAX-XMIN)/NBINS;

  std::vector<double> histo(NBINS, 0.0);
  std::mt19937 gen(SEED);
  std::uniform_real_distribution<double> xu(-1, 1);
  std::uniform_real_distribution<double> yu(0, 3.0/4.0);
  std::ofstream fout("data.txt");
  for (int ii = 0; ii < SAMPLES; ++ii) {
    double x = xu(gen);
    double y = yu(gen);
    if (y < f(x)) {
      fout << x << "\n";
      int bin = int((x - XMIN)/DX);
      if (0 <= bin && bin < NBINS) {
        histo[bin]++;
      }
    }
  }
  fout.close();

  fout.open("histo.txt");
  for (int ii = 0; ii < NBINS; ii++) {
    fout << XMIN + ii*DX << "\t" << histo[ii]/(DX*SAMPLES) << "\n";
  }
  fout.close();

  return 0;
}

```

## Sampling regions

```c++
#include <iostream>
#include <fstream>
#include <cmath>
#include <random>
#include <cstdlib>
#include <vector>

const double SQRT3 = std::sqrt(3);

double f(double x) {
    if (-1 <= x && x <= 0) return SQRT3*x + SQRT3;
    else if (0 < x && x <= 1) return -SQRT3*x + SQRT3;
    else return 0;
}

int main(int argc, char **argv)
{
    const int SEED = std::atoi(argv[1]);;
    const int SAMPLES = std::atoi(argv[2]);

    std::mt19937 gen(SEED);
    std::uniform_real_distribution<double> xu(-1, 1);
    std::uniform_real_distribution<double> yu(0, SQRT3);
    std::ofstream fout("data.txt");
    int samples = 0;
    int counter = 0;
    while (samples < SAMPLES) {
        double x = xu(gen);
        double y = yu(gen);
        if (y < f(x)) {
            fout << x << "\t" << y << "\n";
            samples++;
        }
        counter++;
    }
    fout.close();
    std::cout << "Actual tries: " << counter << "\n";

    return 0;
}

```

## Computing multi-dimensional integrals

```c++
#include <iostream>
#include <fstream>
#include <cmath>
#include <random>
#include <cstdlib>
#include <vector>


double f(double x, double y) {
    return std::sin(std::sqrt(std::log(x+y+1)));
}

bool inregion(double x, double y) {
    return (((x-0.5)*(x-0.5) + (y-0.5)*(y-0.5)) <= 0.25);
}

int main(int argc, char **argv)
{
    const int SEED = std::atoi(argv[1]);;
    const int SAMPLES = std::atoi(argv[2]);

    std::mt19937 gen(SEED);
    std::uniform_real_distribution<double> xu(0, 1);
    std::uniform_real_distribution<double> yu(0, 1);
    std::ofstream fout("data.txt");
    int samples = 0;
    int counter = 0;
    double integral = 0;
    while (samples < SAMPLES) {
        double x = xu(gen);
        double y = yu(gen);
        if (inregion(x,y)) {
            fout << x << "\t" << y << "\n";
            integral += f(x, y);
            samples++;
        }
        counter++;
    }
    fout.close();
    std::cout << "Actual tries: " << counter << "\n";
    std::cout << "Integral: " << integral*M_PI/(4*samples) << "\n";
    return 0;
}

```

## Simulations

### Needle bug

```c++
#include <iostream>
#include <fstream>
#include <cmath>
#include <random>
#include <cstdlib>
#include <vector>

int main(int argc, char **argv)
{
    const int SEED = std::atoi(argv[1]);;
    const int SAMPLES = std::atoi(argv[2]);

    std::mt19937 gen(SEED);
    std::uniform_real_distribution<double> ug(0, 1.0/2.0);
    std::uniform_real_distribution<double> vg(0, M_PI/2.0);
    int samples = 0;
    double count = 0;
    while (samples < SAMPLES) {
        double u = ug(gen);
        double v = vg(gen);
        if (2*u < std::sin(v)) {
            count++;
        }
        samples++;
    }
    std::cout << "Integral: " << count/SAMPLES << "\n";
    return 0;
}

```

### Ising model